## Loading libraries and data

Here we load the necessary libraries. Note that while `pandas`, `numpy`, `sklearn`, `statsmodels` come with Anaconda distribution, you will probably have to install `stargazer` (for printing regression tables) and `plotly` (data visualization) manually.

In [12]:
!pip install --upgrade nbformat>=4.2.0


In [11]:
#@title
# This is to install some packages
!pip install stargazer
!pip install plotly
# You can comment it out later

In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf

from stargazer.stargazer import Stargazer, LineLocation
import plotly.express as px

# Notebook 1: revision of linear regression

Go through the notebook, do exercises, and discuss open questions in the end of the notebook with your group. Later you will share your answers with the whole class and we will discuss them together.

By the end of this activity, you should

1. Get familiar with Jupyter notebooks

2. Review basics of regression modelling

3. Be aware of pitfalls of machine learning - latent variables, biases in training data, overfitting.

4. Explain the importance of setting aside a test dataset

Please run the code chunks one by one, look at the output and make sure that you understand how it is produced. There will be little exercises - please solve them. There will also be discussion questions in the end of this notebook. Please discuss them with your group and submit one answer per group for the class discussion (there will be a WooClap activity for that).

The following commands import the CSV dataset using pandas library and prints the dimensions of the dataset, i.e., the number of rows and the number of columns.

In [3]:
#### Loading data from Google Drive - thanks to ChatGPT

import requests
from io import StringIO

# Set the file ID of the CSV file you want to load
file_id = "1WI7sUJ4EpiKwc2-LQUhQJjUYPricpK5l"

# Set the URL to download the file using the Drive API
url = f"https://drive.google.com/uc?id={file_id}&export=download"

# Make a GET request to download the file and decode the content
content = requests.get(url).content.decode("utf-8")

# Convert the string content to a pandas dataframe
dataset = pd.read_csv(StringIO(content))
dataset.shape

(115302, 11)

Here is what the data look like (it's like an excel spreadsheet):

In [4]:
dataset.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


And some simple description

In [5]:
dataset.describe()

,floor_area_sqm,lease_commence_date,resale_price
count,115302.000000,115302.000000,1.153020e+05
mean,97.850652,1995.040216,4.588013e+05
std,24.136235,13.411162,1.587515e+05
min,31.000000,1966.000000,1.400000e+05
25%,82.000000,1985.000000,3.440000e+05
50%,95.000000,1995.000000,4.280000e+05
75%,113.000000,2005.000000,5.400000e+05
max,249.000000,2019.000000,1.360000e+06


## Data exploration

We will use the library "plotly" for visualization. If you are familiar with "matplotlib", forget about it - it is old and ugly. Switch to some modern library, such as "plotly".

Here is the histogram of  `resale_price`.

In [9]:
fig = px.histogram(dataset, x = 'resale_price', nbins = 30)
fig.update_layout(bargap=0.1)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Let us now explore the relation between `flat_type` and `resale_price` (just as an example of data exploration).

First, we will look at the number of transactions of each flat type.

In [10]:
dataset.flat_type.value_counts()  # this is the pandas analogue of R function "table"

flat_type
4 ROOM              48012
5 ROOM              29334
3 ROOM              26939
EXECUTIVE            9206
2 ROOM               1703
MULTI-GENERATION       60
1 ROOM                 48
Name: count, dtype: int64

Here is the whisker chart showing the distribution of resale price by flat type (it shows the mininimum, the 25%-percentile, the median, the 75%-percentile, the maximum, and the outliers):

In [9]:
fig = px.box(dataset, x="flat_type", y="resale_price")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

This should not be surprising - the price is probably affected by the floor area rather than by the flat type.

Note that the order of flat types is incorrect. Here is how we can change it manually:

In [34]:
fig = px.box(dataset, x="flat_type", y="resale_price",
             category_orders={"flat_type": ["1 ROOM", "2 ROOM", "3 ROOM",\
                                            "4 ROOM", "5 ROOM", "EXECUTIVE" , "MULTI-GENERATION"]})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

### Exercise 1

Plot the distribution of the price by the storey range. Is the flat price increasing or decreasing with the storey range (height above ground)? Is the relationship between the storey range and resale price linear?

In [47]:
### Write your code here


## Data cleaning

Clearly, the two most important predictors for predicting the resale price are the floor area and the remaining lease. The floor area is a numeric variable and its value is given in square metres. The problem with the remaining lease is that its value is given as a string. This is what it looks like:

In [12]:
dataset['remaining_lease'][:6]  # the index [:6] means extracting the first 6 elements of the column
# also note that in Python indexing starts with 0, i.e., the first 6 elements are 0 - 5

0    61 years 04 months
1    60 years 07 months
2    62 years 05 months
3     62 years 01 month
4    62 years 05 months
5              63 years
Name: remaining_lease, dtype: object

Now we will extract the first two characters out of `remaining_lease` (they always represent the whole number of years) and save them as the new variable called `numeric_lease`.

_Note: Fedor is not an expert in data manipulation in Python (he is more familiar with R). If Fedor's code is not optimal, he will be very grateful if you can help to improve this notebook_

In [13]:
# Here, we extract the column 'remaning lease' from dataset
# Then we extract the first two elements of each entry with .str[:2] - here, ".str" is a string operation in pandas
# Then we convert the string containing the first two elements of `remaining lease` to numeric
# by applying .map(pd.to_numeric)
# Finally, "+0.0" is needed to convert an integer to a decimal fraction so that whenever division is applied later,
# 5/2 will be 2.5 rather than 2 with remainder 1.

dataset['numeric_lease'] = dataset['remaining_lease'].str[:2].map(pd.to_numeric) + 0.0
dataset.head(6)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,numeric_lease
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,61.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,60.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,62.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,62.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,62.0
5,2017-01,ANG MO KIO,3 ROOM,150,ANG MO KIO AVE 5,01 TO 03,68.0,New Generation,1981,63 years,275000.0,63.0


In [14]:
# And here is the histogram of the remaining lease

fig = px.histogram(dataset, x = 'numeric_lease', nbins = 30)
fig.update_layout(bargap=0.1)
fig.show()

### Exercise 2

Extract the more precise numeric remaining lease out of the character variable `remaining_lease`. Do it according to the formula

$$
\mbox{Numeric lease} = \mbox{Years} + \frac{\mbox{Months}}{12}
$$

Note that some records do not have the number of months. Extracting symbols 9 and 10 for them will yield `NaN` - you will need to manually change it to 0.

In [15]:
### Code here


## Training and test (validation) datasets

In machine learning, it is extremely important to set aside some part of all the available data. The dataset that the model is actually trained on is called "training data" and the dataset that has been set aside "test data" or "validation data". The test data should not be used in the training process. Its purpose is to give some estimate of the model's performance on data that the model has never seen before.

Here we will split the entire dataset into 80% training and 20% test data (there is no universal recommended rule about proportions of train and test data).

In [16]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state = 8128) # we imported this function
print("Entire dataset dimensions =", dataset.shape)
print("Train data dimensions =", train_data.shape)
print("Test data dimensions =", test_data.shape)

Entire dataset dimensions = (115302, 12)
Train data dimensions = (92241, 12)
Test data dimensions = (23061, 12)


## Regressions

### Univariate model fitting

The simplest linear model is
$$
\mbox{price}=\beta_0+\beta_1\times\mbox{floor area}
$$
Let's fit it first and the look how we can improve it. Below is the summary of the model (it probably contains too much information and we will see how to produce better-looking tables in a few minutes)

Note that we use the module "smf" from the library "statsmodels" because it provides syntax similar to R. If you are used to a different syntax, please fill free to train machine learning models in the manner you prefer.

In [17]:
mod1 = smf.ols(formula = 'resale_price ~ floor_area_sqm', data = train_data).fit()
mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           resale_price   R-squared:                       0.385
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                 5.779e+04
Date:                Mon, 08 Jan 2024   Prob (F-statistic):               0.00
Time:                        11:44:33   Log-Likelihood:            -1.2131e+06
No. Observations:               92241   AIC:                         2.426e+06
Df Residuals:                   92239   BIC:                         2.426e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       5.887e+04   1713.847     34.351      0.000    5.55e+04    6.22e+04
floor_area_sqm  4088.1645     17.006    240.394      0.000    4054.833    4121.496
==============================================================================
Omnibus:                    27322.471   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            77581.766
Skew:                           1.574   Prob(JB):                         0.00
Kurtosis:                       6.206   Cond. No.                         421.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Here are the coefficients of the model:

In [18]:
mod1.params

Intercept         58872.592922
floor_area_sqm     4088.164455
dtype: float64

A linear model is usually easy to interpret (unless it has many inter-correlated variables): the average price of one square metre of an HDB flat is 4096 SGD (rounded to a whole dollar) and the price of owning a flat itself is 58020 SGD (rounded to a whole dollar).

### Model evaluation

To evaluate our model, we need some performance metric. Such a performance metric can either show how good a model is (accuracy) or how bad a model is (error or loss function). For regression models, we usually use a certain loss function and try to find a model minimizing the loss function.

You know that a linear model is trained by minimizing the ordinary least squares loss function (note "OLS" in the summary table above)
$$
E(\beta_0,\beta_1,\cdots,\beta_p)=
\sum_{i=1}^{n}\left(y^i-(\beta_0+\beta_1x^{i}_1+\cdots+\beta_px^{i}_p)\vphantom{\int}\right)^2
$$
It means that, by definition, model parameters are found as to minimize it, i.e.,
$$
(\beta_0,\beta_1,\dots,\beta_p)=\arg\min E
$$
For a linear model, this is done by equating partial derivaives $\frac{\partial E}{\partial \beta_i}$ to $0$ and solving the resulting system of linear equations.

Usually, one scales the OLS loss function by considering the mean squared error rather than the sum of squared errors. It doesn't affect the training process but enables comparison of the error between different datasets. The mean squared error loss is
$$
MSE(\beta_0,\beta_1,\cdots,\beta_p)=
\frac{1}{n}\sum_{i=1}^{n}\left(y^i-(\beta_0+\beta_1x^{i}_1+\cdots+\beta_px^{i}_p)\vphantom{\int}\right)^2
$$

**Important remark:** _in the core of nearly any machine learning algorithm, there is a loss function. Then training a model is minimizing such a loss function. However, usually, it is impossible to find the minimum in a closed form and one resorts to some form of gradient descent to do it. A linear model is one of rare cases when it is possible to find the minimum in a closed form_

Note that while a linear model itself is easy to interpret, the mean squared error loss is not. If we know that the mean squared error is, say, 20K squared singapore dollars, is it a large error? It is much easier to interpret the mean absolute error
$$
MAE(\beta_0,\beta_1,\cdots,\beta_p)=
\frac{1}{n}\sum_{i=1}^{n}\left|y^i-(\beta_0+\beta_1x^{i}_1+\cdots+\beta_px^{i}_p)\vphantom{\int}\right|
$$

Here is how we can do it manually (without built-in functions) just to demonstrate the concepts:

In [19]:
pred_price_1 = mod1.params[0] + mod1.params[1] * test_data['floor_area_sqm']
mae_1 = np.mean(np.abs(pred_price_1 - test_data['resale_price']))
print("Manually calculated error =", mae_1)

Manually calculated error = 89458.49781954488


And now we will do it using the built-in functions:

In [20]:
# First we will calculate the model's predictions
pred_price_1 = mod1.predict(test_data)

# And here is the mean absolute error
mae_1 = metrics.mean_absolute_error(test_data['resale_price'], pred_price_1)
print("Error via built-in function =", mae_1)

Error via built-in function = 89458.49781954488


We will probably want to visualize our model's predictions. To do it, let's create some fake data. Let's imagine that you are looking at a collection of HDB flats that have a remaining lease of 80 years (which is not important now) with floor areas between 80 and 150 square metres. We will create a data frame containing information about such hypothetical HDB flats, use our model to predict their prices, and plot the model's predictions

In [21]:
fake_data = pd.DataFrame(data = {'floor_area_sqm': np.arange(80, 150),
                                 'numeric_lease': 80})
fake_data['predicted_price'] = mod1.predict(fake_data)
print(fake_data.head())
fig = px.line(fake_data, x="floor_area_sqm", y="predicted_price", title='Price depending on area')
fig.show()

   floor_area_sqm  numeric_lease  predicted_price
0              80             80    385925.749357
1              81             80    390013.913812
2              82             80    394102.078268
3              83             80    398190.242723
4              84             80    402278.407179


## Multivariate and polynomial regressions

We will now train three more models (so totally we will have four models).

The second regression is
$$
\mbox{price}=\beta_0+\beta_1\times\mbox{floor area}+\beta_2\times\mbox{remaining lease}
$$
The third regression is
$$
\mbox{price}=\beta_0+\beta_1\times\mbox{floor area}+\beta_2\times\mbox{remaining lease}+
\beta_3\times\mbox{floor area}^2
+\beta_4\times\mbox{remaining lease}^2
$$
The fourth regression is
$$
\mbox{price}=\beta_0+\beta_1\times\mbox{floor area}+\beta_2\times\mbox{remaining lease}+
\beta_3\times\mbox{floor area}^2
+\beta_4\times\mbox{remaining lease}^2+\beta_5\times\mbox{floor area}\times\mbox{remaining lease}
$$
Below we train the model and print one table with all the information about them:

In [22]:
mod2 = smf.ols(formula = 'resale_price ~ floor_area_sqm + numeric_lease', data = train_data).fit()

mod3 = smf.ols(formula = 'resale_price ~ floor_area_sqm + numeric_lease + \
               I(floor_area_sqm**2) + I(numeric_lease**2)', data = train_data).fit()

mod4 = smf.ols(formula = 'resale_price ~ floor_area_sqm * numeric_lease + \
               I(floor_area_sqm**2) + I(numeric_lease**2)', data = train_data).fit()


stargazer = Stargazer([mod1, mod2, mod3, mod4])
stargazer # It is possible to manually update variable names in the first column if you really need it

Now we will report the mean absolute training and test errors (usually we only care about the test error, but sometimes it is useful to look at the training error to understand what's going on) for each of these models

In [23]:
def model_mae(model, data = test_data, y = 'resale_price'):
    pred = model.predict(data)
    return metrics.mean_absolute_error(data[y], pred)

# Here we create a data frame that contains all the info that we need about our four models

error_df = pd.DataFrame(data = {'model': ["(1)", "(2)", "(3)", "(4)"],
                                'description': ["One-var linear", "Two-var linear", "Quadratic", "With mixed term"],
                                'Train MAE': [model_mae(mod1, train_data), model_mae(mod2, train_data),
                                              model_mae(mod3, train_data), model_mae(mod4, train_data)],
                                'Test MAE': [model_mae(mod1), model_mae(mod2), model_mae(mod3), model_mae(mod4)]})

print(error_df)

  model      description     Train MAE      Test MAE
0   (1)   One-var linear  90059.916004  89458.497820
1   (2)   Two-var linear  86938.535638  86136.401904
2   (3)        Quadratic  83155.288102  82781.849600
3   (4)  With mixed term  82975.522180  82612.334225


## Questions for discussion

You will discuss these questions with your group members and report your answer in WooClap. We will then discuss your answer together with the whole class.

### Question 1 - production data

Recall that in order to train our models, we split the entire available data into 80% training and 20% test sets. What if then we decide to use our model in production? In other words, we are going to apply it in a real business situation when new data keeps coming in.

Here we simulate this scenario by loading a new data from a file.

In [48]:
# Set the file ID of the CSV file you want to load
file_id = "1Q7EGClcoNCc2f0mX6RWppgvH-TPdSq-m"

# Set the URL to download the file using the Drive API
url = f"https://drive.google.com/uc?id={file_id}&export=download"

# Make a GET request to download the file and decode the content
content = requests.get(url).content.decode("utf-8")

# Convert the string content to a pandas dataframe
new_data = pd.read_csv(StringIO(content))
new_data['numeric_lease'] = 99.0 - new_data['month'].str[:4].map(pd.to_numeric) + new_data['lease_commence_date']
new_data.head

<bound method NDFrame.head of           month        town  flat_type block       street_name storey_range  \
0       2000-01  ANG MO KIO     3 ROOM   170  ANG MO KIO AVE 4     07 TO 09   
1       2000-01  ANG MO KIO     3 ROOM   174  ANG MO KIO AVE 4     04 TO 06   
2       2000-01  ANG MO KIO     3 ROOM   216  ANG MO KIO AVE 1     07 TO 09   
3       2000-01  ANG MO KIO     3 ROOM   215  ANG MO KIO AVE 1     07 TO 09   
4       2000-01  ANG MO KIO     3 ROOM   218  ANG MO KIO AVE 1     07 TO 09   
...         ...         ...        ...   ...               ...          ...   
369646  2012-02      YISHUN     5 ROOM   212      YISHUN ST 21     10 TO 12   
369647  2012-02      YISHUN     5 ROOM   758      YISHUN ST 72     01 TO 03   
369648  2012-02      YISHUN     5 ROOM   873      YISHUN ST 81     01 TO 03   
369649  2012-02      YISHUN  EXECUTIVE   664      YISHUN AVE 4     07 TO 09   
369650  2012-02      YISHUN  EXECUTIVE   293      YISHUN ST 22     04 TO 06   

        floor_area_sq

In [25]:
print("Test error of model 1 =", model_mae(mod1, test_data))
print("Error of model 1 on new data =", model_mae(mod1, new_data))
print("Test error of model 4 =", model_mae(mod4, test_data))
print("Error of model 4 on new data =", model_mae(mod4, new_data))

Test error of model 1 = 89458.49781954488
Error of model 1 on new data = 177105.13473784406
Test error of model 4 = 82612.3342246848
Error of model 4 on new data = 186800.09249559804


Why are our models so imprecise on the new data?

### Question 2 - predicting future prices

Here we will try to predict future prices using our models. Of course, it doesn't make sense to predict future prices with the first model because it doesn't use the remaining lease as a predictor. But we will try the second (linear) and the fourth (quadratic with the interaction term) models.

First, we create some fake data. We will pretend that we own an HDB flat with area 100 square metres and 50 years of remaining lease as of 2022.

In [26]:
fake_data = pd.DataFrame(data = {'floor_area_sqm': 100,
                                 'numeric_lease': np.arange(50, 0, -1)})
fake_data['year'] = 2072 - fake_data['numeric_lease']
fake_data.head()

,floor_area_sqm,numeric_lease,year
0,100,50,2022
1,100,49,2023
2,100,48,2024
3,100,47,2025
4,100,46,2026


Now we compute the priced predicted by model 2 and the price predicted by model 4. For reference, we will also report the mean absolute error of both models (model 4 is more accurate):

In [27]:
print("Error of model 2 =", model_mae(mod2))
print("Error of model 4 =", model_mae(mod4))

fake_data['predicted_price_2'] = mod2.predict(fake_data)
fake_data['predicted_price_4'] = mod4.predict(fake_data)
fake_data.head()

Error of model 2 = 86136.40190377766
Error of model 4 = 82612.3342246848


,floor_area_sqm,numeric_lease,year,predicted_price_2,predicted_price_4
0,100,50,2022,395546.616686,501363.974824
1,100,49,2023,392618.559697,509736.061387
2,100,48,2024,389690.502708,518539.369064
3,100,47,2025,386762.445719,527773.897856
4,100,46,2026,383834.388730,537439.647764


Note that the prices predicted by the two models are very different. Which of the two is more meaningful? Let's plot predictions by both models

In [28]:
fig = px.line(fake_data, x="year", y="predicted_price_2", title="Future prices predicted by model 2")
fig.show()

In [29]:
fig = px.line(fake_data, x="year", y="predicted_price_4", title="Future prices predicted by model 4")
fig.show()

Note that, by definion of HDB lease, an HDB flat becomes worthless when the remaining lease is 0 - it is not owned, but leased for 99 years. It means that the price of an HDB flat should decline with age. This behaviour is predicted by model 2 (less precise) but not by model 4 (more precise).

Explain why model 4, being more precise (on the test set!), makes nonsensical predictions.

### Question 3

To illustrate a certain important effect in machine learning, we will need to train our models on a small dataset. For the purpose of this discussion, we will create a different split of the data into training and test sets, reserving just 0.1% for the training data.

In [30]:
mini_train_data, maxi_test_data = train_test_split(dataset, test_size=0.999, random_state = 8128)
print("Entire dataset dimensions =", dataset.shape)
print("Train data dimensions =", mini_train_data.shape)
print("Test data dimensions =", maxi_test_data.shape)

Entire dataset dimensions = (115302, 12)
Train data dimensions = (115, 12)
Test data dimensions = (115187, 12)


First, we will train a version of model 2, i.e.,
$$
\mbox{price}=\beta_0+\beta_1\times\mbox{floor area}+\beta_2\times\mbox{remaining lease},
$$
but to do it, we will use a much smaller training set. Below we report the training and test errors of this model. Note that the new training error is a bit smaller than the training error of the original model 2, but the test error is about the same.

In [31]:
mod2a = smf.ols(formula = 'resale_price ~ floor_area_sqm', data = mini_train_data).fit()
print("Train error of model 2a =", model_mae(mod2a, mini_train_data))
print("Test error of model 2a =", model_mae(mod2a, maxi_test_data))

Train error of model 2a = 80695.40313091705
Test error of model 2a = 87506.9438381889


And now we will train a polynomial of degree 6:

In [32]:
mod5 = smf.ols(formula = 'resale_price ~ floor_area_sqm * numeric_lease + \
                I(floor_area_sqm) * I(numeric_lease ** 2) + I(floor_area_sqm**2) * I(numeric_lease) + \
                I(floor_area_sqm) * I(numeric_lease ** 3) + I(floor_area_sqm ** 2) * I(numeric_lease ** 2) +\
                I(floor_area_sqm**3) * I(numeric_lease) + I(floor_area_sqm) * I(numeric_lease ** 4) +\
                I(floor_area_sqm**2) * I(numeric_lease ** 3) + I(floor_area_sqm**3) * I(numeric_lease ** 2) +\
                I(floor_area_sqm) * I(numeric_lease ** 4) + I(floor_area_sqm ** 6) +\
                I(floor_area_sqm) * I(numeric_lease ** 5) + I(floor_area_sqm**2) * I(numeric_lease ** 3) +\
                I(floor_area_sqm**3) * I(numeric_lease ** 3) + I(floor_area_sqm**4) * I(numeric_lease ** 2) +\
                I(floor_area_sqm**5) * I(numeric_lease) + I(numeric_lease ** 6)', data = mini_train_data).fit()

mod5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           resale_price   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     9.111
Date:                Mon, 08 Jan 2024   Prob (F-statistic):           7.11e-15
Time:                        11:44:43   Log-Likelihood:                -1463.8
No. Observations:                 115   AIC:                             2974.
Df Residuals:                      92   BIC:                             3037.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                     2.859e+04   1.73e+04      1.649      0.103   -5843.853     6.3e+04
floor_area_sqm                                3.938e+05   2.45e+05      1.604      0.112   -9.37e+04    8.81e+05
numeric_lease                                 3.247e+05   1.93e+05      1.684      0.095   -5.82e+04    7.07e+05
floor_area_sqm:numeric_lease                 -5.244e+05   2.61e+05     -2.012      0.047   -1.04e+06   -6786.517
I(floor_area_sqm)                             3.935e+05   2.45e+05      1.604      0.112   -9.37e+04    8.81e+05
I(numeric_lease ** 2)                         2.647e+05   1.55e+05      1.712      0.090   -4.23e+04    5.72e+05
I(floor_area_sqm):I(numeric_lease ** 2)       2.149e+04   1.26e+04      1.705      0.092   -3545.270    4.65e+04
I(floor_area_sqm ** 2)                        1.693e+05   9.78e+04      1.732      0.087   -2.49e+04    3.64e+05
I(numeric_lease)                              3.247e+05   1.93e+05      1.684      0.095   -5.82e+04    7.08e+05
I(floor_area_sqm ** 2):I(numeric_lease)      -4489.7241   4422.349     -1.015      0.313   -1.33e+04    4293.443
I(numeric_lease ** 3)                        -1.343e+04   8042.709     -1.670      0.098   -2.94e+04    2545.952
I(floor_area_sqm):I(numeric_lease ** 3)       -372.4548    221.055     -1.685      0.095    -811.489      66.580
I(floor_area_sqm ** 2):I(numeric_lease ** 2)    74.1882     59.315      1.251      0.214     -43.616     191.992
I(floor_area_sqm ** 3)                       -1310.3889    695.540     -1.884      0.063   -2691.792      71.014
I(floor_area_sqm ** 3):I(numeric_lease)          9.0083     18.554      0.486      0.628     -27.841      45.857
I(numeric_lease ** 4)                          252.1681    151.502      1.664      0.099     -48.727     553.064
I(floor_area_sqm):I(numeric_lease ** 4)          2.9607      1.605      1.845      0.068      -0.227       6.148
I(floor_area_sqm ** 2):I(numeric_lease ** 3)    -0.5264      0.296     -1.776      0.079      -1.115       0.062
I(floor_area_sqm ** 3):I(numeric_lease ** 2)    -0.1178      0.271     -0.434      0.665      -0.656       0.421
I(floor_area_sqm ** 6)                        5.203e-05   4.81e-05      1.083      0.282   -4.34e-05       0.000
I(numeric_lease ** 5)                           -2.1031      1.218     -1.727      0.088      -4.522       0.315
I(floor_area_sqm):I(numeric_lease ** 5)         -0.0082      0.004     -1.912      0.059      -0.017       0.000
I(floor_area_sqm ** 3):I(numeric_lease ** 3)     0.0017      0.001      1.741      0.085      -0.000       0.004
I(floor_area_sqm ** 4)                           8.9758      5.963      1.505      0.136      -2.866      20.818
I(floor_area_sqm ** 4):I(numeric

Let's look at the errors of this polynomial model

In [33]:
print("Train error of model 5 =", model_mae(mod5, mini_train_data))
print("Test error of model 5 =", model_mae(mod5, maxi_test_data))

Train error of model 5 = 61189.634545218425
Test error of model 5 = 211892.9064201286


Explain why the training error of this super-complex model is much smaller than that of model 2 but the test error is too large.